In [6]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

a = llm.predict("how many planets are there")
b = chat.predict("how many planets are there")

c:\Users\wmk51\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\wmk51\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [7]:
a,b


('There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

##3-2

In [10]:
from langchain.chat_models import ChatOpenAI
#chatmodel 생성자
chat = ChatOpenAI(temperature=0.1) ##얼마나 창의적인 답변을 요구하느ㄴ냐

In [12]:
from langchain.schema import SystemMessage, AIMessage, HumanMessage
messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in {language}"
    ),
    AIMessage(
        content="Hi my name is {name}!!"
    ),
    HumanMessage(
        content="what is distance between {country_a} and {country_b}, Also, what si your name?"
    ),
]

chat.predict_messages(messages)

AIMessage(content='서울과 부산 사이의 거리는 약 325km입니다. 제 이름은 원호입니다. 어떻게 도와드릴까요?')

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

chat = ChatOpenAI(temperature=0.1)

prompt = template.format(country_a = "Mexico", country_b = "Korea")
chat.predict(prompt)

'The distance between Mexico and South Korea is approximately 8,800 miles (14,200 kilometers) when measured in a straight line.'